In [ ]:
# Skin Cancer Detection Colab Version
# Run this in Google Colab: https://colab.research.google.com/

import gradio as gr
from transformers import AutoModelForImageClassification, AutoImageProcessor
import torch
from PIL import Image
import requests
from io import BytesIO

print("🚀 Loading Skin Cancer Detection Model...")

# Load the model and processor
repo_name = "Jayanth2002/dinov2-base-finetuned-SkinDisease"
try:
    image_processor = AutoImageProcessor.from_pretrained(repo_name)
    model = AutoModelForImageClassification.from_pretrained(repo_name)
    print("✅ Model loaded successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    exit(1)

# Skin cancer focused classes
skin_cancer_classes = {
    'Basal Cell Carcinoma': 'Basal Cell Carcinoma (BCC)',
    'Melanoma': 'Melanoma',
    'squamous cell carcinoma': 'Squamous Cell Carcinoma (SCC)',
    'actinic keratosis': 'Actinic Keratosis (Pre-cancerous)',
    'nevus': 'Benign Mole (No Cancer)',
    'dermatofibroma': 'Benign (No Cancer)',
    'pigmented benign keratosis': 'Benign (No Cancer)',
    'seborrheic keratosis': 'Benign (No Cancer)',
    'vascular lesion': 'Benign (No Cancer)'
}

# Other classes to map to "No Cancer"
other_classes = [
    'Darier_s Disease', 'Epidermolysis Bullosa Pruriginosa', 'Hailey-Hailey Disease',
    'Herpes Simplex', 'Impetigo', 'Larva Migrans', 'Leprosy Borderline',
    'Leprosy Lepromatous', 'Leprosy Tuberculoid', 'Lichen Planus',
    'Lupus Erythematosus Chronicus Discoides', 'Molluscum Contagiosum',
    'Mycosis Fungoides', 'Neurofibromatosis', 'Papilomatosis Confluentes And Reticulate',
    'Pediculosis Capitis', 'Pityriasis Rosea', 'Porokeratosis Actinic',
    'Psoriasis', 'Tinea Corporis', 'Tinea Nigra', 'Tungiasis'
]

original_class_names = ['Basal Cell Carcinoma', 'Darier_s Disease', 'Epidermolysis Bullosa Pruriginosa', 'Hailey-Hailey Disease', 'Herpes Simplex', 'Impetigo', 'Larva Migrans', 'Leprosy Borderline', 'Leprosy Lepromatous', 'Leprosy Tuberculoid', 'Lichen Planus', 'Lupus Erythematosus Chronicus Discoides', 'Melanoma', 'Molluscum Contagiosum', 'Mycosis Fungoides', 'Neurofibromatosis', 'Papilomatosis Confluentes And Reticulate', 'Pediculosis Capitis', 'Pityriasis Rosea', 'Porokeratosis Actinic', 'Psoriasis', 'Tinea Corporis', 'Tinea Nigra', 'Tungiasis', 'actinic keratosis', 'dermatofibroma', 'nevus', 'pigmented benign keratosis', 'seborrheic keratosis', 'squamous cell carcinoma', 'vascular lesion']

def predict(image):
    try:
        # Preprocess the image
        encoding = image_processor(image.convert("RGB"), return_tensors="pt")

        # Get model predictions
        with torch.no_grad():
            outputs = model(**encoding)

        # Get probabilities using softmax
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

        # Get top 5 predictions
        top5_probs, top5_indices = torch.topk(probabilities, 5)

        results = []
        for i in range(5):
            original_class = original_class_names[top5_indices[0][i].item()]
            probability = top5_probs[0][i].item() * 100

            # Map to skin cancer categories
            if original_class in skin_cancer_classes:
                mapped_class = skin_cancer_classes[original_class]
                risk = "HIGH" if any(cancer in mapped_class for cancer in ['Carcinoma', 'Melanoma']) else "MEDIUM" if 'Keratosis' in mapped_class else "LOW"
            else:
                mapped_class = "No Cancer (Other Condition)"
                risk = "LOW"

            results.append({
                'condition': mapped_class,
                'confidence': probability,
                'risk': risk,
                'original': original_class
            })

        # Format the output
        output_text = "🔍 AI SKIN CANCER ANALYSIS RESULTS\n\n"
        output_text += "CONDITION".ljust(35) + "CONFIDENCE".ljust(15) + "RISK\n"
        output_text += "-" * 60 + "\n"

        for result in results:
            icon = "🔴" if result['risk'] == "HIGH" else "🟡" if result['risk'] == "MEDIUM" else "🟢"
            output_text += f"{icon} {result['condition']}".ljust(35)
            output_text += f"{result['confidence']:.1f}%".ljust(15)
            output_text += f"{result['risk']}\n"

        output_text += "\n" + "=" * 60 + "\n"
        output_text += "⚠️ MEDICAL DISCLAIMER:\n"
        output_text += "This AI tool is for educational purposes only.\n"
        output_text += "Always consult a qualified dermatologist for diagnosis."

        return output_text

    except Exception as e:
        return f"❌ Error in analysis: {str(e)}"

# Create Gradio interface
print("🎯 Creating Gradio Interface...")

demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label="📷 UPLOAD SKIN IMAGE"),
    outputs=gr.Textbox(label="📊 AI ANALYSIS RESULTS", lines=15),
    title="🩺 DERMAGUARD AI - SKIN CANCER DETECTOR",
    description="AI-powered detection of Melanoma, Basal Cell Carcinoma, Squamous Cell Carcinoma and other skin conditions",
    examples=[
        "https://raw.githubusercontent.com/ultralytics/ultralytics/main/ultralytics/assets/zidane.jpg"
    ]
)

print("🔗 Launching Server...")
print("👉 The public URL will be generated below:")
demo.launch(share=True)

🚀 Loading Skin Cancer Detection Model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/436 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/347M [00:00<?, ?B/s]

✅ Model loaded successfully!
🎯 Creating Gradio Interface...
🔗 Launching Server...
👉 The public URL will be generated below:
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()


model.safetensors:   0%|          | 0.00/347M [00:00<?, ?B/s]

* Running on public URL: https://6c7d2c3d070f2c3697.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
